In [1]:
using Turing
using LimberJack
using CSV
using NPZ
using Plots
using FITSIO
using Plots
using LinearAlgebra
using Distributed

In [2]:
Threads.nthreads()

12

In [10]:
[i Threads.@threads(i for i in 1:5)]

LoadError: LoadError: ArgumentError: @threads requires a `for` loop expression
in expression starting at In[10]:1

In [ ]:
files = npzread("Cls_meta.npz")
Cls_meta = cls_meta(files)

In [ ]:
cov_tot = files["cov"]
data_vector = files["cls"]

In [ ]:
heatmap(1:size(cov_tot,1), 1:size(cov_tot,2), log.((cov_tot.^2).^0.5))

In [ ]:
@model function model(data_vector)
    Ωm ~ Uniform(0.2, 0.3)
    h ~ Uniform(0.6, 0.8)
    s8 ~ Uniform(0.7, 1.0)
    
    b0 ~ Uniform(1.0, 3.0)
    b1 ~ Uniform(1.0, 3.0)
    b2 ~ Uniform(1.0, 3.0)
    b3 ~ Uniform(1.0, 3.0)
    b4 ~ Uniform(1.0, 3.0)
    
    nuisances = Dict("b0" => b0,
                     "b1" => b1,
                     "b2" => b2,
                     "b3" => b3,
                     "b4" => b4)
    
    cosmology = LimberJack.Cosmology(Ωm, 0.05, h, 0.96, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    theory = Theory(cosmology, nuisances, Cls_meta, files).Cls
    data_vector ~ MvNormal(theory, cov_tot)
end;

iterations = 10

# Start sampling.
folname = string("DES_full_test_", "MH")
if isdir(folname)
    println("Folder already exists")
    if isfile(joinpath(folname, "chain.jls"))
        println("Restarting from past chain")
        past_chain = read(joinpath(folname, "chain.jls"), Chains)
        new_chain = sample(model(data_vector), MH(), iterations,
                           progress=true; save_state=true, resume_from=past_chain)
    else
        new_chain = sample(model(data_vector), MH(),
                    iterations, progress=true; save_state=true)
    end
else
    mkdir(folname)
    println("Created new folder")
    new_chain = sample(model(data_vector), MH(),
                iterations, progress=true; save_state=true)
end

summary = describe(new_chain)[1]
fname_summary = string("summary.csv")
CSV.write(joinpath(folname, fname_summary), summary)

fname_jls = string("chain.jls")
write(joinpath(folname, fname_jls), new_chain)
    
fname_csv = string("chain.csv")
CSV.write(joinpath(folname, fname_csv), new_chain)

